In [1]:
import pickle
from Experiment_Burgers_1 import *

,value
param,
seed,1
method,RK3
decay_const,9.000e-01
train_split,2.500e-01
m,4
epochs,30000
l_wd,1.000e-07
fs,7
neurons,32


In [2]:
import sys
# sys.path.append(fr'/home/mex0/Documents/TukhbatullinAL/diplom_2/Git_stencil-net/Git_stencil-net/')

sys.path.append('D:\myProjects\diplom_2\GIT_stencil-net\GIT_stencil-net')

from utils import forcing_terms

In [3]:
BC_TYPE='dirichlet'; BC_VALUES=[0,0]

### Описание:

Зная гипперпараметры, обучаю на большом числе эпох

### 0. Параметры для генерации данных

In [ ]:
# folder='Experiment_burgers_1_optuna'

# optimal_params_filename=fr"best_params_n=256_[[5], 16, 3, 0.01, 1, False, 'E1', 0.9, 1000].xlsx"

In [9]:
# folder='Experiment_burgers_1_optuna_2'
# folder='Experiment_burgers_1_optuna_3'
# folder='Experiment_burgers_1_optuna_4'
# folder='Experiment_burgers_1_optuna_5'
# folder='Experiment_burgers_1_optuna_6'
# folder='Experiment_burgers_1_optuna_7'

folder='Experiment_burgers_1_optuna'
folder_read='Experiment_burgers_1'

In [5]:
# для сил
np.random.seed(0)

L = 2.0*np.pi; T_ = 40;  Al = -0.1; Ar = 0.1; wl = -0.4; wr = 0.4; N = 20
A = np.zeros((N,)); w = np.zeros((N,)); phi = np.zeros((N,)); l = np.zeros((N,))

for k in range(0, N):
    A[k] = (Ar-Al)*np.random.rand(1) + Al
    w[k] = (wr-wl)*np.random.rand(1) + wl
    phi[k] = (2.0*np.pi)*np.random.rand(1)
    l[k] = np.random.randint(2, 5)

In [34]:
epochs_lst=[1000]

# new_epoch_lst=[20_000]
new_epoch_lst=[None]

In [7]:
display(main_params)

,value
param,
seed,1
method,RK3
decay_const,9.000e-01
train_split,2.500e-01
m,4
epochs,30000
l_wd,1.000e-07
fs,7
neurons,32


### Читаю данные и нужные гипперпараметры

In [11]:
# n = 256; Tsim = 80001;
# Lx = 101; Tsim = 10001;
# n = 64; Tsim = 2001;

# n = 256; Tsim = 16001; tau=0.01

n = 64; Tsim = 8001;tau = 0.01; D=0.1

x_lst = np.linspace(0,L,n); h = x_lst[1] - x_lst[0]; #D = 0.02;

CFL = (h*h)/(2.0*D)
print("CFL dt", CFL, "actual dt", tau)

v_fact = pickle.load( open(fr'{folder_read}/data/v_fact_CD_n={n}_Tsim={Tsim}_D={D}.pkl', 'rb')).to_device('cpu')

CFL dt 0.04973345629170752 actual dt 0.01


In [24]:
optimal_params_filename="collab_output/all_best_params_n=64_[7, 32, 2, 4, 'E1', 1000].xlsx"
optimal_params=pd.read_excel(fr'{optimal_params_filename}')
optimal_params.set_index('Parameter',inplace=True)
optimal_params


,Value
Parameter,
fs,7
neurons,32
hidden_layers_num,2
act_func,ELU(alpha=1.0)
epoch,1000
lr,1.000e-03
tau,1.000e-02
h,9.973e-02
n,64


### Замена исходных параметров на оптимальные

In [26]:
neurons=optimal_params.loc['neurons'][0]
fs=optimal_params.loc['fs'][0]
hidden_layers_num=optimal_params.loc['hidden_layers_num'][0]
lr=optimal_params.loc['lr'][0]
method=optimal_params.loc['method'][0]
decay_const	= optimal_params.loc['decay_const'][0]
m	= optimal_params.loc['m'][0]
has_backward=bool(optimal_params.loc['has_backward'][0])

### Сабсемплинг

In [27]:
t_factor = 1  # time subsample factor 
s_factor = 1  # space subsample factor


Tsim=len(v_fact[0])
dxc,dtc,coarse_t,coarse_x,v_coarse,Lxc,\
Ltc,v_coarse_train,v_coarse_test=subsampling(s_factor,t_factor,h,tau,Tsim,n,v_fact,train_split=train_split)

full_sample (64, 8001)
train (64, 2000)
test (64, 6001)


### Обучение

In [28]:

Fc, Fc_0p5, Fc_p1, Fc_0m5, Fc_m1 = forcing_terms(A, w, phi, l, L, Lxc, T_, Ltc, N, dtc)

# u_train = torch.tensor(v_coarse_train.T, requires_grad=True, dtype=torch.float, device=device)
fc      = torch.tensor(Fc.T, requires_grad=True, dtype=torch.float, device=device)
fc_0p5  = torch.tensor(Fc_0p5.T, requires_grad=True, dtype=torch.float, device=device)
fc_p1   = torch.tensor(Fc_p1.T, requires_grad=True, dtype=torch.float, device=device)
fc_0m5  = torch.tensor(Fc_0m5.T, requires_grad=True, dtype=torch.float, device=device)
fc_m1   = torch.tensor(Fc_m1.T, requires_grad=True, dtype=torch.float, device=device)

In [ ]:
# for epochs in epochs_lst:
    
#     net, loss_lst,loss=train_net(MLPConv,v_coarse_train,epochs,dtc,
#               fs,
#                     neurons,
#                     hidden_layers_num,
#                     lr,
#                     m,
#                     has_backward,
#                     method,
#                     decay_const,
#               force_terms=[fc,fc_0p5,fc_p1,fc_0m5,fc_m1],
#               bc_type=BC_TYPE,bc_values=BC_VALUES)
    
#     pickle.dump(net, open(fr'{folder}/models/net_epochs={epochs}', 'wb'))
    

In [ ]:
# for i in range(len(epochs_lst)):
#     epoch=epochs_lst[i]
#     if new_epoch_lst==[None]:
#         print('starting study')
#         net, loss_lst,loss=train_net(MLPConv,v_coarse_train,epoch,dtc,
#                     fs,
#                     neurons,
#                     hidden_layers_num,
#                     lr,
#                     m,
#                     has_backward,
#                     method,
#                     decay_const,
#                     force_terms=[fc,fc_0p5,fc_p1,fc_0m5,fc_m1],
#                     bc_type=BC_TYPE,bc_values=BC_VALUES,
#                     verbose=True,
#                     verbose_step=1000,
#                     continue_fitting=None)
#         pickle.dump(net, open(fr'{folder}/models/net_epochs={epoch}', 'wb'))
#     else:
#         print('continue study')
#         new_epoch=new_epoch_lst[i]
#         net = pickle.load(open(fr'{folder}/models/net_epochs={epoch}', 'rb'))
#         net, loss_lst,loss=train_net(MLPConv,v_coarse_train,new_epoch-epoch,dtc,
#                     fs,
#                     neurons,
#                     hidden_layers_num,
#                     lr,
#                     m,
#                     has_backward,
#                     method,
#                     decay_const,
#                     force_terms=[fc,fc_0p5,fc_p1,fc_0m5,fc_m1],
#                     bc_type=BC_TYPE,bc_values=BC_VALUES,
#                     verbose=True,
#                     verbose_step=1000,
#                     continue_fitting=[net,new_epoch])
        
#         pickle.dump(net, open(fr'{folder}/models/cf_net_epochs={new_epoch}', 'wb'))

In [ ]:
# epochs_lst=new_epoch_lst

### Предикт

In [50]:
NN_sim_lst=[]

weights=[]
biases=[]

for epochs in epochs_lst:
    
    # net = pickle.load(open(fr'{folder}/models/net_epochs={epochs}', 'rb'))

    # net=torch.load(open(fr'{folder}/models/net_epochs={epochs}', 'rb'),
    #                weights_only=False,
    #                map_location='cpu'
    #                )

    net=pickle.load(open(fr'{folder}/models/net_epochs={epochs}', 'rb')
                    )
    
    # NN_sim,T_sim,x_sim=make_simulation(net,v_coarse,L,Lxc,dtc,method=method)
    NN_sim,T_sim,x_sim=make_simulation_forcing(net,v_coarse,L,Lxc,dtc,\
                                              A,phi,N,w,l,method=method,
                                              bc_type=BC_TYPE,bc_values=BC_VALUES)
    
    NN_sim_lst.append(NN_sim)
    
    weights.append(net.layer[0].weight.data.numpy())
    biases.append(None)

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [44]:
device

device(type='cpu')

### Графики

In [ ]:
T=round(T_sim*dtc,3)
T

In [ ]:

for i in range(len(epochs_lst)):
    
    print(fr'---------------Epochs = {epochs_lst[i]}-------------------')
    # view_results(T_sim,x_sim,NN_sim_lst[i],v_coarse,T,dtc,n=11,fix_axes=True,
    #             epochs=epochs_lst[i],
    #             save_path = fr'{folder}/graphs/',
    #             save_name = fr'Result_epoch={epochs_lst[i]}',
    #             view_flag=False)
    view_result_metric(NN_sim_lst[i],v_coarse,T,dtc,L,dxc,
                       n_xticks=2000,
                       save_path = fr'{folder}/graphs/',
                save_name = fr'Metric_epoch={epochs_lst[i]}')

    view_result_imshow(NN_sim_lst[i],v_coarse,T,dtc,L,dxc,
                       figsize=(10,5),aspect=10,n_xticks=1001,n_yticks=40,
                       save_path = fr'{folder}/graphs/',
                       save_name = fr'Imshow_epoch={epochs_lst[i]}',
                       fix_colorbar_axes=False,
                       colorbar_min_max=[-0.1,1.1],cmap='seismic'
                       )
    
    # make_gif(folder=fr'{folder}/graphs/',epochs=epochs_lst[i])

    # make_gif(folder=fr'{folder}/graphs/',epochs=epochs_lst[i],format='mp4')

    make_subplot_graphs(NN_sim_lst[0], v_coarse, x_sim, T_sim, T, dtc,16,nx=4,ny=4,
                        figsize=(30,30),
                        save_flg=True,
                        save_path = fr'{folder}/graphs/',
                        save_name=fr'SUBPLOT_epoch={epochs_lst[i]}')

    print('\n')
print('done')